In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp1').getOrCreate()
print('Spark is Connected')

24/01/12 13:11:40 WARN Utils: Your hostname, MaheshPC resolves to a loopback address: 127.0.1.1; using 192.168.1.7 instead (on interface enp5s0)
24/01/12 13:11:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/12 13:11:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/01/12 13:11:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/01/12 13:11:42 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Spark is Connected


In [4]:
# importing data 

data = spark.read.csv('SMSSpamCollection',inferSchema=True,sep='\t')
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [5]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [6]:
from pyspark.sql.functions import *

data = data.withColumn('length',length(data['text']))
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [7]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [15]:
from pyspark.ml.feature import *

tokenizer = Tokenizer(inputCol='text',outputCol='token_text')
remover = StopWordsRemover(inputCol='token_text',outputCol='stop_token')
cv = CountVectorizer(inputCol='stop_token',outputCol='cv_token')
idf = IDF(inputCol='cv_token',outputCol='tf_idf')

res = StringIndexer(inputCol='class',outputCol='label')



In [16]:
assembler = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')


In [17]:
from pyspark.ml.classification import *
nb=NaiveBayes()


In [18]:
from pyspark.ml import Pipeline

data_pipe = Pipeline(stages=[res,tokenizer,remover,cv,idf,assembler])

In [19]:
cl = data_pipe.fit(data)
clean_data = cl.transform(data)

In [20]:
clean_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token',
 'cv_token',
 'tf_idf',
 'features']

In [22]:
df = clean_data.select('label','features')
df.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,140...|
|  0.0|(13424,[10,53,102...|
|  0.0|(13424,[127,185,4...|
|  1.0|(13424,[1,47,121,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,117...|
|  1.0|(13424,[8,16,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,95,221...|
|  0.0|(13424,[555,1797,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,44...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [24]:
train,test = df.randomSplit([0.7,0.3])


In [25]:
spamdect = nb.fit(train)
test_res = spamdect.transform(test)


24/01/12 13:31:43 WARN DAGScheduler: Broadcasting large task binary with size 1166.6 KiB
24/01/12 13:31:44 WARN DAGScheduler: Broadcasting large task binary with size 1146.0 KiB


In [26]:
test_res.show()

24/01/12 13:31:53 WARN DAGScheduler: Broadcasting large task binary with size 1371.6 KiB


+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-791.48746216273...|[1.0,2.1011619005...|       0.0|
|  0.0|(13424,[0,1,2,13,...|[-606.49436918096...|[1.0,2.3244442763...|       0.0|
|  0.0|(13424,[0,1,5,20,...|[-797.51175863499...|[1.0,5.5173714868...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-1160.5820954258...|[1.0,8.8300600590...|       0.0|
|  0.0|(13424,[0,1,11,32...|[-867.35611637268...|[1.0,2.8727584114...|       0.0|
|  0.0|(13424,[0,1,16,19...|[-807.90631142888...|[1.0,2.2419232127...|       0.0|
|  0.0|(13424,[0,1,18,20...|[-826.44914838085...|[1.0,3.2526305966...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-970.47827740062...|[0.99999999999999...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-994.55744159165...|[1.0,1.7126281861...|       0.0|
|  0.0|(13424,[0

In [27]:
from pyspark.ml.evaluation import *

m_eval = MulticlassClassificationEvaluator()
acc = m_eval.evaluate(test_res)
acc

24/01/12 13:33:13 WARN DAGScheduler: Broadcasting large task binary with size 1376.6 KiB


0.9276394133922778